<a href="https://colab.research.google.com/github/rybread1/fashion-mnist/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2

In [2]:
pip install albumentations --upgrade

Requirement already up-to-date: albumentations in /usr/local/lib/python3.6/dist-packages (0.4.6)


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
folder = '/gdrive/My Drive/colab-notebooks/fashion-mnist'

train = pd.read_csv(os.path.join(folder, 'fashion-mnist_train.csv'))
test = pd.read_csv(os.path.join(folder, 'fashion-mnist_test.csv'))

In [5]:
train_labels, train_px = train.iloc[:, 0].values, train.iloc[:, 1:].values
test_labels, test_px = test.iloc[:, 0].values, test.iloc[:, 1:].values

train_px = np.array([ex.reshape(28, 28, 1)/255.0 for ex in train_px])
test_px = np.array([ex.reshape(28, 28, 1)/255.0 for ex in test_px])

In [6]:
import albumentations as A

img_augmenter = A.Compose(
        [
          A.augmentations.transforms.Cutout(num_holes=4, max_h_size=2, max_w_size=2, fill_value=0, p=0.5),
          A.augmentations.transforms.Cutout(num_holes=4, max_h_size=2, max_w_size=2, fill_value=1, p=0.5),
          A.augmentations.transforms.HorizontalFlip(p=0.5),
        ], 
        p=1
    )

In [7]:
aug_train_px = [img_augmenter(image=img)['image'] for img in train_px]

x2 = np.stack((np.array(aug_train_px), train_px)).reshape(120000, 28, 28, 1)
y2 = np.concatenate((train_labels,train_labels))

In [15]:
input = tf.keras.layers.Input(shape=(28, 28, 1),)
x = tf.keras.layers.Conv2D(256, (3,3), activation='relu')(input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(512, (3,3), activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(1024, (3,3), activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024)(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(1024)(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(train['label'].nunique(), activation='softmax')(x)

model = tf.keras.Model(input, output)

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 26, 26, 256)       2560      
_________________________________________________________________
batch_normalization_8 (Batch (None, 26, 26, 256)       1024      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 11, 11, 512)       1180160   
_________________________________________________________________
batch_normalization_9 (Batch (None, 11, 11, 512)      

In [17]:
model.fit(x=train_px,
          y=train_labels,
          validation_data=(test_px, test_labels), 
          batch_size=32, 
          epochs=5,
          shuffle=True,)

Epoch 1/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2302 - accuracy: 0.9155 - val_loss: 0.2763 - val_accuracy: 0.9014
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2249 - accuracy: 0.9167 - val_loss: 0.2582 - val_accuracy: 0.9105
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2170 - accuracy: 0.9197 - val_loss: 0.2509 - val_accuracy: 0.9117
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2080 - accuracy: 0.9234 - val_loss: 0.2587 - val_accuracy: 0.9126
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2032 - accuracy: 0.9254 - val_loss: 0.2526 - val_accuracy: 0.9116
